In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get -qq update
apt-get -qq install chromium chromium-driver

# Install selenium
pip install --quiet selenium requests beautifulsoup4
pip install --quiet tranformers
pip install --quiet langchain openai
pip install --quiet python-dotenv sentence_transformers chromadb tiktoken accelerate

Executing: /tmp/apt-key-gpghome.WL9BNNeT1T/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.5vFg5KwNnc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.kHo2lbafQA/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Selecting previously unselected package udev.
(Reading database ... 120493 files and directories cur

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
BASE_URL = 'https://docs.vmware.com/en/VMware-Tanzu-Kubernetes-Grid/2.2/using-tkg-22/'
URL = BASE_URL + 'workload-index.html'
MAX_CHUNK = 1000

service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36'
options.add_argument('user-agent={0}'.format(user_agent))
browser = webdriver.Chrome(service=service, options=options)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [6]:
def soup_by_url(url):
  browser.get(url)
  # Selenium hands the page source to Beautiful Soup
  return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
def find_linked_pages(soup):
  links = []
  for tag in soup.find_all(['a']):
    if tag.get('href') not in ['#', '/', '', None]:
      links.append(tag.get('href'))

  filtered = filter(lambda link: 'vmware.com' not in link
                    and '.pdf' not in link
                    and '.com' not in link
                    and '.io' not in link
                    and '/' not in link
                    and 'javascript' not in link
                    and 'workload-index.html' not in link, links)

  return list(dict.fromkeys(list(filtered)))

In [8]:
def extract_page_content(soup):
  results = soup.find_all(['h1', 'p', 'ul', 'li', 'pre', 'button'])
  page_text = [result.text for result in results]
  return ' '.join(page_text)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 2000,
    chunk_overlap  = 100,
    add_start_index = True,
)

In [10]:
from langchain.vectorstores import Chroma
db = Chroma(persist_directory="./content/chroma_db", embedding_function=embeddings, collection_name="tanzu")
print(f"Documents in the Vector DB: {db._collection.count()}")
db.delete()
db.persist()

Documents in the Vector DB: 0


In [11]:
soup = soup_by_url(URL)
links = find_linked_pages(soup)
print(f"Linked pages: {links}")
documents = []
for link in links:
  soup = soup_by_url(BASE_URL + link)
  page_content = extract_page_content(soup)
  docs = text_splitter.create_documents([page_content])
  print(f"Page: {link}; Content len: {len(page_content)}; # of chunks: {len(docs)}")
  db.add_documents(docs)

db.persist()

Linked pages: ['install-cli.html', 'workload-clusters-index.html', 'workload-clusters-deploy.html', 'workload-clusters-configure.html', 'workload-clusters-configure-vsphere.html', 'workload-clusters-configure-vsphere-with-tanzu.html', 'workload-clusters-configure-aws.html', 'workload-clusters-configure-azure.html', 'workload-clusters-legacy-cc.html', 'workload-clusters-node-networking.html', 'workload-clusters-networking.html', 'workload-clusters-storage.html', 'workload-clusters-k8s-versions.html', 'workload-clusters-advanced-aws.html', 'workload-clusters-advanced-azure.html', 'workload-clusters-advanced-vsphere.html', 'workload-clusters-hardware.html', 'workload-clusters-multi-az-vsphere.html', 'workload-clusters-tiny.html', 'workload-clusters-ytt.html', 'workload-clusters-cclass.html', 'workload-clusters-manage-index.html', 'workload-clusters-examine.html', 'workload-clusters-scale.html', 'workload-clusters-pool.html', 'workload-clusters-secret.html', 'workload-clusters-mhc.html', '

In [12]:
query = "How to create a Tanzu workload cluster node pool?"

In [13]:
retriever = db.as_retriever(search_type="mmr")
results = retriever.get_relevant_documents(query)

In [19]:
len(results)
results[0]

Document(page_content='tanzu cluster create my-cluster --file my-cluster-config.yaml --dry-run > my-cluster-spec.yaml\n You can then use this object spec to deploy a cluster as described in Create a Class-Based Cluster from the Object Spec. Proceed to Create Workload Clusters.                                            Company                                                               Support                    \n×\n Anyone with the Link can view', metadata={'start_index': 39936})

In [14]:
#end the Selenium browser session
browser.quit()

In [15]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end.
Let's work this out in a step by step way to be sure we have the right answer.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [16]:
from transformers import AutoTokenizer
import transformers
import torch
from langchain import HuggingFacePipeline

model = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=4000
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=PROMPT, llm=llm)

context = ' '.join([page_content for doc in results])

print(llm_chain.run({'context':context, 'question':query}))